1 IMPORTAR LIBRERIAS

Nuestro Input(x) va a ser los comentarios(review)
Nuestro Output(y) va a ser los sentimientos(sentiment)

In [ ]:
import pandas as pd
import imblearn as imb
import sklearn as sk

ESTE DATASET TIENE 2 COLUMNAS "review" Y "sentiment" y 50000 Filas

In [ ]:
df_review = pd.read_csv("IMDB Dataset.csv")
df_review

Pero para entrenar este modelo que vamos a desarrollar vamos a usar 9000 reseñas positivas y 1000 negativas para desbalancearla

In [ ]:
df_positivo = df_review[df_review["sentiment"] == "positive"] [:9000]
df_negativo = df_review[df_review["sentiment"] == "negative"] [:1000]

creamos un nuevo datset desbalanceado

In [ ]:
df_review_desbalanceado = pd.concat([df_positivo, df_negativo])
df_review_desbalanceado.value_counts("sentiment")

DATASET DESBALANCEADO

LO que vamos a hacer es balancear la data con una libreria "imbLearn"
Usaremos undersampling para disminuir la Data con mayor informacion para bajarla al mismo nivel que la otra

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
df_review_balanceado, df_review_balanceado["sentiment"] = rus.fit_resample(df_review_desbalanceado[["review"]], df_review_desbalanceado["sentiment"])
df_review_balanceado.value_counts(["sentiment"])

Ahora lo que vamos a hacer es separar la data para entrenar (train) y la data para testear (test)

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_review_balanceado, test_size=0.33, random_state=42)
train_x, train_y = train["review"], train["sentiment"] 
test_x, test_y = test["review"], test["sentiment"]

Con esto tenemos asignadas el input y el output para entrenar y testear

REPRESENTACION DE TEXTO (BAGS OF WORDS)

- CountVectorizer = La frecuencia en la cual una palabra aparece en una oracion
- Tfidf = es la relevancia que tiene una palabra dentro de una oracion que no este repetida en los otros reviews

Transformar data de texto a data numerica

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
Tfdif = TfidfVectorizer(stop_words="english")
train_x_vector = Tfdif.fit_transform(train_x)
test_x_vector = Tfdif.transform(test_x)

Ahora usaremos MACHINE LEARNING

Usaremos Aprendizaje Supervisado -> Clasificacion -> SVM, Arbol de decisión, Naive Bayes, Regresión Logística

SUPPOR VECTOR MACHINES (SVM)

In [ ]:
from sklearn.svm import SVC 
svc = SVC(kernel="linear")
svc.fit(train_x_vector, train_y)

Testeo (SVM)

In [ ]:
print(svc.predict(Tfdif.transform(["This movie is amazing"])))
print(svc.predict(Tfdif.transform(["This movie is terrible"])))
print(svc.predict(Tfdif.transform(["I did not like this movie at all"])))

ARBOL DE DECISION

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
dec_tree.fit(train_x_vector, train_y)

NAIVE BAYES

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(train_x_vector.toarray(), train_y)

REGRESION LOGISTICA

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_x_vector, train_y)

EVALUACION DE MODELOS

Score (Accuracy)

In [ ]:
print(svc.score(test_x_vector, test_y))
print(dec_tree.score(test_x_vector, test_y))
print(gnb.score(test_x_vector.toarray(), test_y))
print(lr.score(test_x_vector, test_y))

Queda evidenciado que el modelo de evaluacion svc quedo con mayor porcentaje de precision(0.8545454545454545)

F1 Score

F1 Score = 2(Recall * Precission) / (Recall + Precission)

In [ ]:
from sklearn.metrics import f1_score
f1_score(test_y, svc.predict(test_x_vector), labels=["positive", "negative"], average=None)

Reporte de Clasificación

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_y, svc.predict(test_x_vector), labels=["positive", "negative"]))

Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, svc.predict(test_x_vector), labels=["positive", "negative"])

- [TruePositives  FalsePositives]
- [FalseNegatives  TrueNegatives]

Optimizacion del modelo

In [ ]:
from sklearn.model_selection import GridSearchCV
parametros = {"C":[1,4,8,16,32], "kernel":["linear", "rbf",]}
svc = SVC()
clf = GridSearchCV(svc, parametros)
svc_grid = GridSearchCV(svc, parametros, cv=5)
svc_grid.fit(train_x_vector, train_y)

In [ ]:
print(svc_grid.best_estimator_)
print(svc_grid.best_params_)
print(svc_grid.best_score_)